In [1]:
import pandas as pd

In [2]:
source = '../source/eci_country_rankings.csv'

In [3]:
data = pd.read_csv(source)

In [4]:
data.head()

,Year,Country,Country ID,ECI,ECI+
0,1964,Angola,afago,-1.317530,-1.226750
1,1964,Cote d'Ivoire,afciv,-0.915735,-1.028210
2,1964,Cameroon,afcmr,-1.128520,-0.889755
3,1964,Republic of the Congo,afcog,-0.930207,-0.898081
4,1964,Algeria,afdza,-0.131416,0.090527


In [11]:
countries = data[['Country ID', 'Country']].drop_duplicates().copy()

In [12]:
countries.columns = ['country', 'name']

In [28]:
countries.to_csv('../../ddf--entities--country.csv', index=False)

In [16]:
dps = data[['Country ID', 'Year', 'ECI', 'ECI+']].copy()

In [17]:
dps.columns = ['country', 'year', 'eci', 'eci_plus']

In [18]:
dps = dps.set_index(['country', 'year'])

In [20]:
dps.head()

,,eci,eci_plus
country,year,,
afago,1964,-1.317530,-1.226750
afciv,1964,-0.915735,-1.028210
afcmr,1964,-1.128520,-0.889755
afcog,1964,-0.930207,-0.898081
afdza,1964,-0.131416,0.090527


In [24]:
for c in dps:
    dps[[c]].to_csv(f'../../ddf--datapoints--{c}--by--country--year.csv')

In [23]:
!open ../../

In [25]:
concepts = [
    {
        'concept': 'country',
        'name': 'Country',
        'concept_type': 'entity_domain',
        'domain': ''
    },
    {
        'concept': 'name',
        'name': 'Name',
        'concept_type': 'string',
        'domain': ''
    },
    {
        'concept': 'year',
        'name': 'Year',
        'concept_type': 'time',
        'domain': ''
    },
    {
        'concept': 'eci',
        'name': 'ECI',
        'concept_type': 'measure',
        'domain': ''
    },
    {
        'concept': 'eci_plus',
        'name': 'ECI+',
        'concept_type': 'measure',
        'domain': ''
    }
]

cdf = pd.DataFrame.from_records(concepts)

In [26]:
cdf

,concept,concept_type,domain,name
0,country,entity_domain,,Country
1,name,string,,Name
2,year,time,,Year
3,eci,measure,,ECI
4,eci_plus,measure,,ECI+


In [27]:
cdf.to_csv('../../ddf--concepts.csv', index=False)

In [29]:
!ddf create_datapackage --update ../../

backing up previous datapackage.json...
2018-06-19 13:32:55 WARNING no resources or ddfSchema in datapackage.json
2018-06-19 13:32:55 WARNING skipping file datapackage.json
2018-06-19 13:32:55 WARNING skipping file README.md
2018-06-19 13:32:55 WARNING skipping file .gitignore
2018-06-19 13:32:55 WARNING skipping file .gitattributes
2018-06-19 13:32:55 WARNING skipping file datapackage.json.bak
2018-06-19 13:32:55 INFO generating ddf schema, may take some time...
2018-06-19 13:32:55 INFO loading dataset from disk: ddf--mit--economic_complexity_rankings
100%|█████████████████████████████████████████████| 4/4 [00:00<00:00, 47.04it/s]
Done.


In [ ]:
!validate-ddf ../../

7root checking   0% /                                                           87root checking 100%[==========================================================>]87dataset loading   0% /                                                         8
Validation was finished with issues. Details are here: validation-2018-06-19T053304.518Z.log.
7dataset loading 100%[========================================================>]8